In [1]:
!pip install diffusers transformers scipy ftfy accelerate flask_ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.2 MB/s eta 0:00:00


In [7]:
!pip install pyngrok==4.1.1

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=29a0f27f9101dfb08b95084a5ab61f1c6ab95248c5d60fbb41e68a53130cd5c0
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [8]:
!ngrok authtoken 2TMDR8vu6AYH9u72C9oeSEOimvb_6S9M7rPaffxnXaJmkVKw9

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [9]:
import os
if not os.path.isdir('/content/app'):
  os.mkdir("/content/app")
os.chdir("/content/app")

In [10]:
if not os.path.isdir('/content/app/static'):
  os.mkdir("./static")
os.chdir("./static")

css = """
    body {
    font-family: Arial, Helvetica, sans-serif;
}

form {
    /* Center the form on the page */
    margin: 0 auto;
    width: 400px;
    /* Form outline */
    padding: 1em;
    display: flex;
}

img {
    border-radius: 8px;
    display: block;
    margin-left: auto;
    margin-right: auto;
    margin-top: 40px;
    filter: drop-shadow(5px 5px 10px #000);
    opacity: 1;
    hover: opacity: 0.6;
    transition: opacity 1s ease 0s;

}

ul,
li {
    display: inline
}

ul {
    list-style: none;
    padding: 0;
    margin: 0;
    display: flex
}

form li+li {
    margin-top: 1em;
}

label {
    /* Uniform size & alignment */
    display: inline-block;
    width: 90px;
    text-align: right;
}

input,
textarea {
    /* To make sure that all text fields have the same font settings
         By default, textareas have a monospace font */
    font: 1em sans-serif;

    /* Uniform text field size */
    width: 300px;
    box-sizing: border-box;

    /* Match form field borders */
    border: 1px solid #999;
}

input:focus,
textarea:focus {
    /* Additional highlight for focused elements */
    border-color: #000;
}

textarea {
    /* Align multiline text fields with their labels */
    vertical-align: top;

    /* Provide space to type some text */
    height: 5em;
}

.button {
    /* Align buttons with the text fields */
    padding-left: 90px;
    /* same size as the label elements */
}

button {
    /* This extra margin represent roughly the same space as the space
         between the labels and their text fields */
    margin-left: 0.5em;
}

label,
input {
    display: flex;
    flex-direction: column;
}

input,
label {
    display: block;
}"""

with open("styles.css", 'w') as f:
  f.write(css)

os.chdir("..")

In [11]:
import base64
s = b'iVBORw0KGgoAAAANSUhEUgAAABkAAAAZCAYAAADE6YVjAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAJcEhZcwAADsMAAA7DAcdvqGQAAANrSURBVEhL3ZVLbE1BGMdn5lwuomk9bmlpUq82IqQSEQuiYUMIISJtUlytUo+FR4J2USJCSBOU0Gooi9K0Wo3HRiwECdqIsvCsPrS5dFNEvBbnjP83M+fqde9py9L/Zs73nTkz32+++eZm2H8jbqyn8oq+Ztq2U+w4zijbtgfBcsexme04PY5t364/M36fGeqpPiGByTtGJo+f9mJS+tzEuPgkBggDRFmAJflouxvKU0rMlJgSxnpIbgp1Pg3cvVUqm5tqGed6TWRdHyoy1lOekMCU7cOYlFlSOkwyh4U6mmXjvQv4ooJLehhQwvKN73LJ8ZInhEsxH1GmU1DOLTwE+9jT1f7zx5dS3fc7KzB3qRcPeUO42IxQBEDzSTTqLfH74/YgboceFdbUpRva0owfpZiQMemFaYxbC9CQgM80qxPgqkvHR/3AXu2nBHRtaIZaTqGaHEMxIVh9EFGH6SxUJui1zne/PvpJfZesAc/vxKDR1AdlLclrHWf8CEVBkqYeSEAGBb0ByOQTHuVmCLt8IvAZpoJwQKApjh/M9eT8qehMuMhH8AQTHO+w3KrrfnXogxmhhLCHXY8YfR2ACMi4GUcGI2C2yQLzfExQPbh1zgwJq7o0sRsDrukkNAg2fknu2yB5vRWZibTmo8AZGuKTlAXag9DzfQ/NiD8kqwigpUFSsgLTEVYEBMG3EIBOFRMA6VNVZj5Hqebk2FqY9ohTxtnsxcGWCfTmKgxJyTg1EYCF6tjqTMiGYK+bITGFoAeNB59IilZMD1dhCAKuw6qHK4CuA5qvouvZLnVs+1A9gjvGJyi11YvWtYw2XRqSOuscTpO11QTWjY6tsC6qUX2o9lQSLeKMu2Umm6Gcy/BxNpmINQCMVMeVaqFhNzufbGvX3/vVYRVbbVV42/ZSD0mkZlQKBA2qQvfKBLXBn21gqjudHMKEO242uixsxOLgmxxyhPAPycTqZ+rgBiR8j7lt3VNDB65KNxv10y9rVc+kOTVXcF+spAPOcG/QTSGZzG9rzIn6A/anFQVdrbgpU+l6dhxExc2JaBMFtmWe2R69TcJ6j2zq9LS/1lFt9H6ZzFYJbI3frYUBnW19lN3fsfWQrMY2fdO1IYKifEfR+TJA7qMuTQDsBOgYffkXXS1LocXNJReMl4h9gzFW9Qt/ieYwvJpjQQAAAABJRU5ErkJggg=='
with open('/content/app/static/icon.png', 'wb') as f:
  f.write(base64.decodebytes(s))

In [12]:
html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" type="text/css" href="{{ url_for('static',filename='styles.css') }}" />
    <link rel="icon" type="image/x-icon" href="{{ url_for('static',filename='icon.png') }}" />
    <title>Stable Diffusion App</title>
</head>
<body>
  <h1>Stable Diffusion App</h1>
    <p>Generate images with stable diffusion for free! Enter your prompt below and hit "Create" to generate an image (takes ~10 seconds per image).</p>
    <p>Companion article: <a href="https://www.assemblyai.com/blog/build-a-free-stable-diffusion-app-with-a-gpu-backend/">Build a free Stable Diffusion app with a GPU backend</a> on the <href="https://www.assemblyai.com/blog/">AssemblyAI Blog</a>.</p>
    <form action="\submit-prompt" method="post" id="submission-form">
        <ul>
            <li>
              <label for="prompt">Prompt:</label>
              <input type="text" id="prompt" name="prompt-input" />
            </li>
            <li class="button">
                <button type="submit">Create</button>
            </li>
          </ul>
    </form>
    <img src="{{ generated_image|default("https://images.squarespace-cdn.com/content/v1/6213c340453c3f502425776e/0715034d-4044-4c55-9131-e4bfd6dd20ca/2_4x.png?format=512w", true) }}" class="mainImg" alt="Generated Image">
</body>
</html>

"""

with open("index.html", 'w') as f:
  f.write(html)


In [13]:
import os
os.chdir("/content/app")

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, send_file, request

import torch
from diffusers import StableDiffusionPipeline

import base64
from io import BytesIO

# Load model
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", revision="fp16", torch_dtype=torch.float16)
pipe.to("cuda")

# Start flask app and set to ngrok
app = Flask(__name__, template_folder='.') # period is because default templates folder is /templates
run_with_ngrok(app)

@app.route('/')
def initial():
  return render_template('index.html')

@app.route('/submit-prompt', methods=['POST'])
def generate_image():
  prompt = request.form['prompt-input']
  print(f"Generating an image of {prompt}")

  image = pipe(prompt).images[0]
  print("Image generated! Converting image ...")

  buffered = BytesIO()
  image.save(buffered, format="PNG")
  img_str = base64.b64encode(buffered.getvalue())
  b = "data:image/png;base64," + str(img_str)[2:-1]

  print("Sending image ...")
  return render_template('index.html', generated_image=b)

app.run()

vae/diffusion_pytorch_model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]